In [1]:
import pandas as pd

In [15]:
# Dataset 1 => daily fuel

In [2]:
# Loading data
df = pd.read_csv("eia_rto_daily_fuel_2022_full.csv")

In [3]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710143 entries, 0 to 710142
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   period                710143 non-null  object
 1   respondent            710143 non-null  object
 2   respondent-name       710143 non-null  object
 3   fueltype              710143 non-null  object
 4   type-name             710143 non-null  object
 5   timezone              710143 non-null  object
 6   timezone-description  710143 non-null  object
 7   value                 710143 non-null  int64 
 8   value-units           710143 non-null  object
dtypes: int64(1), object(8)
memory usage: 48.8+ MB


In [5]:
# Convert date column
df['period'] = pd.to_datetime(df['period'])

In [6]:
# Filter 2022 (Precautionary)
df = df[df['period'].dt.year == 2022]

In [7]:
# Keeping only 1 timezone "Eastern"
df = df[df['timezone'] == 'Eastern']

In [8]:
# Renaming and Cleaning
df = df.rename(columns={
    'type-name': 'fuel',
    'value': 'generation_MWh'
})


In [9]:
# Aggregate to U.S. daily generation
us_daily_generation = (
    df
    .groupby(['period', 'fuel'])['generation_MWh']
    .sum()
    .reset_index()
)

In [10]:
us_daily_generation.head()

,period,fuel,generation_MWh
0,2022-01-01,Coal,5773854
1,2022-01-01,Hydro,2424090
2,2022-01-01,Natural Gas,8630241
3,2022-01-01,Nuclear,6915615
4,2022-01-01,Other,541869


In [11]:
# Data for 01/01/2022
us_daily_generation[us_daily_generation['period'] == '2022-01-01']

,period,fuel,generation_MWh
0,2022-01-01,Coal,5773854
1,2022-01-01,Hydro,2424090
2,2022-01-01,Natural Gas,8630241
3,2022-01-01,Nuclear,6915615
4,2022-01-01,Other,541869
5,2022-01-01,Petroleum,62073
6,2022-01-01,Solar,576708
7,2022-01-01,Wind,4457331


In [12]:
us_daily_generation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2920 entries, 0 to 2919
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   period          2920 non-null   datetime64[ns]
 1   fuel            2920 non-null   object        
 2   generation_MWh  2920 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 68.6+ KB


In [76]:
us_daily_generation.head(10)

,period,fuel,generation_MWh
0,2022-01-01,Coal,5773854
1,2022-01-01,Hydro,2424090
2,2022-01-01,Natural Gas,8630241
3,2022-01-01,Nuclear,6915615
4,2022-01-01,Other,541869
5,2022-01-01,Petroleum,62073
6,2022-01-01,Solar,576708
7,2022-01-01,Wind,4457331
8,2022-01-02,Coal,6689943
9,2022-01-02,Hydro,2398782


In [77]:
# Pivot: fuel types become columns
pivot_df = df.pivot_table(
    index='period',
    columns='fuel',
    values='generation_MWh',
    aggfunc='sum'   # sum in case duplicates exist
).reset_index()

In [78]:
# Clean column names
pivot_df.columns.name = None
pivot_df = pivot_df.rename(columns={
    'Coal': 'coal_MWh',
    'Hydro': 'hydro_MWh',
    'Natural Gas': 'natural_gas_MWh',
    'Nuclear': 'nuclear_MWh',
    'Solar': 'solar_MWh',
    'Wind': 'wind_MWh',
    'Petroleum': 'petroleum_MWh',
    'Other': 'other_MWh'
})

In [79]:
# Sort by date
pivot_df = pivot_df.sort_values('period')

In [80]:
pivot_df.head(10)

,period,coal_MWh,hydro_MWh,natural_gas_MWh,nuclear_MWh,other_MWh,petroleum_MWh,solar_MWh,wind_MWh
0,2022-01-01,5773854,2424090,8630241,6915615,541869,62073,576708,4457331
1,2022-01-02,6689943,2398782,10963404,6915756,536574,64398,616311,3024690
2,2022-01-03,7357572,2628471,13300701,6951003,596037,132255,660924,3295071
3,2022-01-04,7070958,2580795,12196374,6960789,636387,75651,662832,5424765
4,2022-01-05,7162800,2437491,11525235,6930051,615591,60951,641382,4855872
5,2022-01-06,7905612,2546895,12629241,6951411,615180,74397,689916,3778842
6,2022-01-07,8801688,2585454,13206267,6878616,636663,153312,713202,3858801
7,2022-01-08,8009769,2485047,10370664,6849495,634758,156132,684144,5182182
8,2022-01-09,7374810,2297001,9576414,6821598,599442,125070,583110,4513656
9,2022-01-10,8736141,2541459,12710478,6869670,656859,168330,633498,2574891


In [82]:
pivot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   period           365 non-null    datetime64[ns]
 1   coal_MWh         365 non-null    int64         
 2   hydro_MWh        365 non-null    int64         
 3   natural_gas_MWh  365 non-null    int64         
 4   nuclear_MWh      365 non-null    int64         
 5   other_MWh        365 non-null    int64         
 6   petroleum_MWh    365 non-null    int64         
 7   solar_MWh        365 non-null    int64         
 8   wind_MWh         365 non-null    int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 25.8 KB


In [81]:
# Storing data
us_daily_generation.to_csv("us_daily_generation_different_sources_2022.csv", index=False)

In [16]:
# Dataset 2 => daily_region

In [54]:
# Loading data
df2 = pd.read_csv("eia_rto_daily_region_2022_full.csv")

In [58]:
# Convert period to datetime
df2['period'] = pd.to_datetime(df2['period'])

In [59]:
# Keep only 2022
df2 = df2[df2['period'].dt.year == 2022]

In [60]:
# Remove duplicate timezone rows
df2 = df2.drop_duplicates(subset=['period', 'respondent', 'type', 'value'])

In [62]:
# Remove timezone duplication
df2 = df2[df2['timezone'] == 'Eastern']

In [65]:
# Aggregate across ALL utilities/respondents
daily_sum = (
    df2.groupby(['period', 'type-name'])['value']
    .sum()
    .reset_index()
)

In [69]:
daily_sum.head(10)

,period,type-name,value
0,2022-01-01,Day-ahead demand forecast,29622237
1,2022-01-01,Demand,29830935
2,2022-01-01,Net generation,29383223
3,2022-01-01,Total interchange,-177129
4,2022-01-02,Day-ahead demand forecast,31110270
5,2022-01-02,Demand,31720815
6,2022-01-02,Net generation,31237195
7,2022-01-02,Total interchange,-290674
8,2022-01-03,Day-ahead demand forecast,35110353
9,2022-01-03,Demand,35464533


In [70]:
daily_pivot = daily_sum.pivot(
    index='period',
    columns='type-name',
    values='value'
).reset_index()

In [71]:
# Rename columns to match required output
daily_pivot = daily_pivot.rename(columns={
    'Demand Forecast': 'us_demand_forecast_MWh',
    'Demand': 'us_demand_MWh',
    'Net Generation': 'us_generation_MWh',
    'Total Interchange': 'us_interchange_MWh'
})


In [72]:
# Sort by date
daily_pivot = daily_pivot.sort_values('period')

In [73]:
daily_pivot.head(10)

type-name,period,Day-ahead demand forecast,us_demand_MWh,Net generation,Total interchange
0,2022-01-01,29622237,29830935,29383223,-177129
1,2022-01-02,31110270,31720815,31237195,-290674
2,2022-01-03,35110353,35464533,34926384,-318716
3,2022-01-04,35335932,35931893,35622266,-155446
4,2022-01-05,34239734,34560132,34262977,-128312
5,2022-01-06,35548867,35625171,35162889,-263182
6,2022-01-07,36745008,37290030,36840482,-265038
7,2022-01-08,34266405,34777923,34314553,-223905
8,2022-01-09,31977264,32316612,31943397,-202326
9,2022-01-10,35309568,35362080,34902209,-271092


In [74]:
daily_pivot.tail()

type-name,period,Day-ahead demand forecast,us_demand_MWh,Net generation,Total interchange
360,2022-12-27,36746089,36748813,36364096,-292916
361,2022-12-28,34537558,34863129,34497454,-283892
362,2022-12-29,32490189,32070666,31621633,-384025
363,2022-12-30,30955977,30589992,30112164,-329508
364,2022-12-31,29313821,29349481,29098879,-113124


In [75]:
# Save final dataset
daily_pivot.to_csv("us_daily_energy_2022.csv", index=False)

In [83]:
daily_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   period                     365 non-null    datetime64[ns]
 1   Day-ahead demand forecast  365 non-null    int64         
 2   us_demand_MWh              365 non-null    int64         
 3   Net generation             365 non-null    int64         
 4   Total interchange          365 non-null    int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 14.4 KB
